In [ ]:
!pip install pytorch-adapt

### Helper function for demo

In [ ]:
from pytorch_adapt.utils.common_functions import get_lr


def print_optimizers_slim(adapter):
    for k, v in adapter.optimizers.items():
        print(f"{k}: {v.__class__.__name__} with lr={get_lr(v)}")

### Adapters Initialization

Models are usually the only required argument when initializing adapters. Optimizers are created using the default that is defined in the adapter. 

In [ ]:
import torch

from pytorch_adapt.adapters import DANN
from pytorch_adapt.containers import Models

G = torch.nn.Linear(1000, 100)
C = torch.nn.Linear(100, 10)
D = torch.nn.Sequential(torch.nn.Linear(100, 1), torch.nn.Flatten(start_dim=0))
models = Models({"G": G, "C": C, "D": D})

adapter = DANN(models=models)
print_optimizers_slim(adapter)

### Modifying optimizers using the Optimizers container

We can use the Optimizers container if we don't want to use the defaults.

For example: SGD with lr 0.1 for all 3 models

In [ ]:
from pytorch_adapt.containers import Optimizers

optimizers = Optimizers((torch.optim.SGD, {"lr": 0.1}))
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

SGD with lr 0.1 for the G and C models only. The default optimizer will be used for D.

In [ ]:
optimizers = Optimizers((torch.optim.SGD, {"lr": 0.1}), keys=["G", "C"])
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

SGD with lr 0.1 for G, and SGD with lr 0.5 for C

In [ ]:
optimizers = Optimizers(
    {"G": (torch.optim.SGD, {"lr": 0.1}), "C": (torch.optim.SGD, {"lr": 0.5})}
)
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

You can also create the optimizers yourself and pass them into the Optimizers container

In [ ]:
optimizers = Optimizers({"G": torch.optim.SGD(G.parameters(), lr=0.123)})
adapter = DANN(models=models, optimizers=optimizers)
print_optimizers_slim(adapter)

### Adapters Training Step

In [ ]:
from pytorch_adapt.utils import common_functions as c_f

device = torch.device("cuda")
adapter.models.to(device)

data = {
    "src_imgs": torch.randn(32, 1000),
    "target_imgs": torch.randn(32, 1000),
    "src_labels": torch.randint(0, 10, size=(32,)),
    "src_domain": torch.zeros(32),
    "target_domain": torch.zeros(32),
}

data = c_f.batch_to_device(data, device)
loss = adapter.training_step(data)

### Adapters Inference

In [ ]:
data = torch.randn(32, 1000).to(device)
features, logits = adapter.inference(data)